## 11e December

# Text Processing

- Text processing
    - NLP - Natural Language Processing
- Olika typer av NLP
    - Classification
        - vad är textens ämne?
    - Censur/Filtrering
        - Does this text contain abuse
    - Sentiment analasys
        - Är texten positiv eller negativ?
    - LLM
        - Vad bör vara det nästa ordet i meningen? (generativ)
    - Översättning
        - Hur säger man "du är vacker" på Urdu?
    - Summering
        - Summera texten till en kort paragraf, tack.

### Hur processar vi text för neurala nätverk?

- Standardisering
    - .lower()
    - ta bort skiljetecken (men försiktigt då de är viktiga)
- Tokenisering och tensorisering
    - Splitta texten till tokens, där varje ord eller grupp av ord blir omgjorda till vektorer, alla blir en tensor
    - Skapar ett index över vektorerna och vi använder sedan indexet.
- **Exempel**
    - "The cat sat on the mat!"
    - the cat sat on the mat
    - "cat", "sat", "on", "mat"
    - [2, 34, 53, 8]
    - onehot encoding eller embedding

- Tre sätt att hantera tokens: (finns fler)
    - **word-level tokenization**
        - Mellanslagsseparerade substrings -> ord typ.
        - Finns varianter som splittar ord som funkar bra i språk som har sammanslagna ord. (Bildörr -> bil dörr)
            - eller om man vill splitta ut ändelser "mädchen"
            - Man behöver alltså ha koll på språket man hanterar!
    - **N-grams**
        - Tokens är grupper av *N* antal ord.
        - Exempel: "the cat", "he was", "over there"
        - 2-grams eller bigrams (grekiska prefix, "hexagram")
            - "bag of words"
    - **Character-level tokenization**
        - Varje bokstav tokeniseras för sig.
        - Användbart för typ cyrilliska/kinesiska/japanska eller fonetiska språkanalyser

Dagens dataset
https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [1]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("../data/aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    try:
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        num_vaL_samples = int(0.2 * len(files))
        val_files = files[:num_vaL_samples]
        for fname in val_files:
            shutil.move(train_dir / category / fname, val_dir / category / fname)
    except FileExistsError:
        pass

In [2]:
import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    val_dir, batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    base_dir / "test", batch_size=batch_size
)

Found 22136 files belonging to 2 classes.
Found 2864 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [10]:
for inputs, targets in train_ds:
    print("inputs:", inputs[:10])
    print("targets:", targets[:10])
    print("inputs shape:", inputs.shape, inputs.dtype)
    print("targets shape:", targets.shape, targets.dtype)
    break

inputs: tf.Tensor(
[b"all i can say is i fully expected the tavern wench/lady pirate to start removing all her clothes at every turn, as i have seen this caliber of acting before... in another genre. +++SPOILER+++ this is NOT a porn movie! tho it would make more sense if it was. i was about to wager that Asylum Home Entertainment was a company trying to break in to mainstream producing, when i noticed comments elsewhere that it appears their M.O. is to 'piggy-back' blockbuster movies by releasing 'C' movies with similar titles. this certainly would explain this release. i know better now. watch this expecting a 0... and u will be pleasantly surprised with a 1 by the way... why hasn't the producer or the grips' mother written in the lead review yet? usually they get it in fairly fast."
 b"I'm sure that writer-director Mike Sarne thought he was developing a new cinematic paradigm when he was making Joanna, but the end result is less than satisfying. Genevieve Waite's over-the-top perform

In [4]:
text_vectorization = keras.layers.TextVectorization(
    max_tokens=20_000, output_mode="multi_hot"
)

text_only_train_ds = train_ds.map(lambda x, _: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [12]:
def get_model(max_tokens=20000, hidden_dims=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = keras.layers.Dense(hidden_dims, activation="relu")(inputs)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [13]:
model = get_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)
]

model.fit(binary_1gram_train_ds.cache(), validation_data=binary_1gram_val_ds.cache(), epochs=10, callbacks=callbacks)

Epoch 1/10


1527/2255 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.1400 - loss: -116.2164

KeyboardInterrupt: 

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.8812 - loss: 0.3101
Test acc: 0.882


## term frequency–inverse document frequency

In [15]:
text_vectorization = keras.layers.TextVectorization(
    ngrams=2, max_tokens=20_000, output_mode="tf_idf"
)

text_vectorization.adapt(text_only_train_ds)

tfidf_2grams_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2grams_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2grams_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
model = get_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2grams.keras", save_best_only=True)
]

In [ ]:
model.fit(tfidf_2grams_train_ds.cache(), validation_data=tfidf_2grams_val_ds.cache(), epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - accuracy: 0.6930 - loss: 0.6772 - val_accuracy: 0.8686 - val_loss: 0.3222
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8502 - loss: 0.3584 - val_accuracy: 0.8832 - val_loss: 0.3110
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8733 - loss: 0.3067 - val_accuracy: 0.8884 - val_loss: 0.3152
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8787 - loss: 0.2836 - val_accuracy: 0.8734 - val_loss: 0.3426
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8879 - loss: 0.2640 - val_accuracy: 0.8794 - val_loss: 0.3526
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8994 - loss: 0.2456 - val_accuracy: 0.8634 - val_loss: 0.3766
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8982 - loss: 0.2394 - val_accuracy: 0.8416 - val_loss: 0.4002
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9005 - loss: 0.2372 - val_accuracy: 

In [ ]:
model = keras.models.load_model("tfidf_2grams.keras")
print(f"Test acc: {model.evaluate(tfidf_2grams_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.8813 - loss: 0.3034
Test acc: 0.879


## Sekvensera texten, embedding

In [5]:
max_length = 600
max_tokens = 20_000
text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens, output_mode="int", output_sequence_length=max_length
)
text_vectorization.adapt(text_only_train_ds)

In [6]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("int_sequence.keras", save_best_only=True)
]

In [ ]:
# import tensorflow as tf

# inputs = keras.Input(shape=(None,), dtype="int64")
# embedded = tf.one_hot(inputs, depth=max_tokens)
# x = keras.layers.Bidirectional(keras.layers.LSTM(32))(embedded)
# x = keras.layers.Dropout(0.5)(x)
# outputs = keras.layers.Dense(1, activation="sigmoid")(x)
# model = keras.Model(inputs, outputs)
# model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)

In [7]:

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = keras.layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(32))(embedded)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  5,120,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 64)        │     73,984 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embedding_bidir_gru.keras", save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
347/692 ━━━━━━━━━━━━━━━━━━━━ 1:28 257ms/step - accuracy: 0.6316 - loss: 0.6239

KeyboardInterrupt: 